# ControlD API tool use

Claude tool use with ControlD API endpoints.

In [34]:
from anthropic import Anthropic
import requests
from typing import Dict, List, Optional
import os
from datetime import date
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Anthropic client with API key
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
if not anthropic_api_key:
    raise ValueError("Please set ANTHROPIC_API_KEY environment variable")
    
client = Anthropic(api_key=anthropic_api_key)
MODEL_NAME = "claude-3-5-sonnet-20241022"

In [35]:
def get_controld_profiles(auth_token: str) -> Dict:
    """List all profiles"""
    headers = {
        'accept': 'application/json',
        'authorization': f'Bearer {auth_token}'
    }
    
    response = requests.get(
        'https://api.controld.com/profiles',
        headers=headers
    )
    
    response.raise_for_status()
    return response.json()

def create_controld_profile(auth_token: str, name: str, clone_profile_id: Optional[str] = None) -> Dict:
    """Create a new profile or clone an existing one"""
    headers = {
        'accept': 'application/json',
        'authorization': f'Bearer {auth_token}',
        'content-type': 'application/x-www-form-urlencoded'
    }
    
    data = {'name': name}
    if clone_profile_id:
        data['clone_profile_id'] = clone_profile_id
    
    try:
        response = requests.post(
            'https://api.controld.com/profiles',
            headers=headers,
            data=data
        )
        
        # Print debug info (will be redacted by our redaction function)
        print(f"Request URL: https://api.controld.com/profiles")
        print(f"Request Headers: {headers}")
        print(f"Request Data: {data}")
        print(f"Response Status: {response.status_code}")
        print(f"Response Content: {response.text}")
        
        response.raise_for_status()
        return response.json()
        
    except requests.exceptions.RequestException as e:
        print(f"API Error: {str(e)}")
        return {"error": str(e)}

In [36]:
tools = [
    {
        "name": "get_profiles",
        "description": "Retrieves all profiles associated with a ControlD account",
        "input_schema": {
            "type": "object",
            "properties": {
                "auth_token": {
                    "type": "string",
                    "description": "ControlD API authentication token"
                }
            },
            "required": ["auth_token"]
        }
    },
    {
        "name": "create_profile",
        "description": "Creates a new ControlD profile or clones an existing one",
        "input_schema": {
            "type": "object",
            "properties": {
                "auth_token": {
                    "type": "string",
                    "description": "ControlD API authentication token"
                },
                "name": {
                    "type": "string",
                    "description": "Name of the new profile"
                },
                "clone_profile_id": {
                    "type": "string",
                    "description": "The exact profile ID (PK) to clone from. Must be the full ID string, not just the profile name."
                }
            },
            "required": ["auth_token", "name"]
        }
    }
]

In [37]:
# Add this function at the beginning of Cell 4
def redact_sensitive_info(text: str, auth_token: str) -> str:
    """Redact sensitive information from output"""
    # Redact auth token
    if auth_token in text:
        text = text.replace(auth_token, "[REDACTED_TOKEN]")
    
    # Redact any Bearer token format
    text = text.replace(f"Bearer {auth_token}", "Bearer [REDACTED]")
    
    return text

def chat_with_controld(user_query: str, auth_token: str):
    """
    Chat interface that allows querying and managing ControlD profiles through Claude
    """
    messages = [{
        "role": "user", 
        "content": f"Auth Token: {auth_token}\n\nUser Query: {user_query}"
    }]
    
    system_prompt = """
    You are an assistant that helps users interact with their ControlD profiles.
    You have access to the following tools:
    1. get_profiles - Lists all existing profiles
    2. create_profile - Creates a new profile or clones an existing one
    
    The auth token will be provided in the user's message, prefixed with "Auth Token: ".
    Always extract and use this token when calling any tools.
    
    When users want to create a profile:
    - Always ask for the name they want to give the new profile if not provided
    - Ask if they want to clone an existing profile
    - If they want to clone, first use get_profiles to show available profiles
    """
    
    response = client.messages.create(
        system=system_prompt,
        model=MODEL_NAME,
        max_tokens=1000,
        messages=messages,
        tools=tools,
        tool_choice={"type": "auto"}
    )
    
    last_content_block = response.content[-1]
    # Replace the followup response section in Cell 4
    if last_content_block.type == "tool_use":
        print("Calling ControlD API...")
        try:
            if last_content_block.name == "get_profiles":
                result = get_controld_profiles(auth_token)
            elif last_content_block.name == "create_profile":
                result = create_controld_profile(
                    auth_token=auth_token,
                    name=last_content_block.input["name"],
                    clone_profile_id=last_content_block.input.get("clone_profile_id")
                )
                print(f"API Response: {redact_sensitive_info(str(result), auth_token)}")
            
            # Send the result back to Claude for interpretation
            followup_response = client.messages.create(
                system=system_prompt,
                model=MODEL_NAME,
                max_tokens=1000,
                messages=[
                    *messages,
                    {"role": "assistant", "content": f"I called the {last_content_block.name} API."},
                    {"role": "user", "content": f"Here is the response: {result}"}
                ]
            )
            print("\nClaude's interpretation of the response:")
            print(redact_sensitive_info(followup_response.content[-1].text, auth_token))
            
        except Exception as e:
            print(f"Error calling ControlD API: {str(e)}")

In [38]:
# Get auth token
controld_token = os.getenv("CONTROLD_API_TOKEN")
if not controld_token:
    raise ValueError("Please set CONTROLD_API_TOKEN environment variable")

# Test queries
queries = [
    "Create a new profile called 'work', and clone from 'relatives'",
]

for query in queries:
    print(f"\nQuery: {query}")
    print("-" * 50)
    chat_with_controld(query, controld_token)


Query: Create a new profile called 'work', and clone from 'relatives'
--------------------------------------------------
Calling ControlD API...

Claude's interpretation of the response:
I can see that the 'relatives' profile exists with PK '628525otprmu'. I'll help you create a new profile called 'work' by cloning from 'relatives'.

Let me call the create_profile API with these parameters:
- name: "work"
- clone_from: "628525otprmu" (the PK of the 'relatives' profile)

Let me make that API call now.
